In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker #For changing ticker location and formatting
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [3]:
#Load in clean CSV
clean_test = pd.read_csv('./datasets/clean_recoded_test_ames.csv', index_col = 'id')
clean_test.head()
    #I'm at the end of the worflow, but I just realized I always should have been reading in the data files by making the 'id' the index column.

,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,...,misc_val,mo_sold,yr_sold,sale_type,age_at_sale,age_since_remod,bsmnt_fin_sf,sold_in_crisis,has_wood_deck,has_pool
id,,,,,,,,,,,,,,,,,,,,,
2658,902301120,190,RM,69.0,9142,Pave,Grvl,4,Lvl,AllPub,...,0,4,2006,WD,96,56,0,0,0,0
2718,905108090,90,RL,0.0,9662,Pave,NaN,3,Lvl,AllPub,...,0,8,2006,WD,29,29,0,0,1,0
2414,528218130,60,RL,58.0,17104,Pave,NaN,3,Lvl,AllPub,...,0,9,2006,New,0,0,554,0,1,0
1989,902207150,30,RM,60.0,8520,Pave,NaN,4,Lvl,AllPub,...,0,7,2007,WD,84,1,0,0,0,0
625,535105100,20,RL,0.0,9500,Pave,NaN,3,Lvl,AllPub,...,0,7,2009,WD,46,46,609,1,0,0


In [11]:
clean_test = clean_test.select_dtypes(exclude = 'object')
clean_test.head(5)

,ms_subclass,lot_frontage,lot_area,lot_shape,land_slope,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,...,pool_area,fence,misc_val,yr_sold,age_at_sale,age_since_remod,bsmnt_fin_sf,sold_in_crisis,has_wood_deck,has_pool
id,,,,,,,,,,,,,,,,,,,,,
2658,190,69.0,9142,4,3,6,8,1910,1950,0.0,...,0,0,0,2006,96,56,0,0,0,0
2718,90,0.0,9662,3,3,5,4,1977,1977,0.0,...,0,0,0,2006,29,29,0,0,1,0
2414,60,58.0,17104,3,3,7,5,2006,2006,0.0,...,0,0,0,2006,0,0,554,0,1,0
1989,30,60.0,8520,4,3,5,6,1923,2006,0.0,...,0,0,0,2007,84,1,0,0,0,0
625,20,0.0,9500,3,3,6,5,1963,1963,247.0,...,0,0,0,2009,46,46,609,1,0,0


In [5]:
clean_test.dtypes

pid                  int64
ms_subclass          int64
lot_frontage       float64
lot_area             int64
lot_shape            int64
                    ...   
age_since_remod      int64
bsmnt_fin_sf         int64
sold_in_crisis       int64
has_wood_deck        int64
has_pool             int64
Length: 63, dtype: object

In [12]:
pd.set_option('display.max_rows', 70)
clean_test.isnull().sum()

ms_subclass        0
lot_frontage       0
lot_area           0
lot_shape          0
land_slope         0
overall_qual       0
overall_cond       0
year_built         0
year_remod/add     0
mas_vnr_area       0
exter_qual         0
exter_cond         0
bsmt_qual          0
bsmt_cond          0
bsmt_exposure      0
bsmtfin_type_1     0
bsmtfin_sf_1       0
bsmtfin_type_2     0
bsmtfin_sf_2       0
bsmt_unf_sf        0
total_bsmt_sf      0
heating_qc         0
central_air        0
electrical         0
1st_flr_sf         0
2nd_flr_sf         0
low_qual_fin_sf    0
gr_liv_area        0
bsmt_full_bath     0
bsmt_half_bath     0
full_bath          0
half_bath          0
bedroom_abvgr      0
kitchen_abvgr      0
kitchen_qual       0
totrms_abvgrd      0
functional         0
fireplaces         0
fireplace_qu       0
garage_finish      0
garage_cars        0
garage_area        0
garage_qual        0
garage_cond        0
paved_drive        0
wood_deck_sf       0
open_porch_sf      0
enclosed_porc

In [10]:
cols_to_drop = ['pid', 'garage_yr_blt', 'mo_sold']
clean_test = clean_test.drop(cols_to_drop, axis = 1)

In [13]:
clean_test.head()

,ms_subclass,lot_frontage,lot_area,lot_shape,land_slope,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,...,pool_area,fence,misc_val,yr_sold,age_at_sale,age_since_remod,bsmnt_fin_sf,sold_in_crisis,has_wood_deck,has_pool
id,,,,,,,,,,,,,,,,,,,,,
2658,190,69.0,9142,4,3,6,8,1910,1950,0.0,...,0,0,0,2006,96,56,0,0,0,0
2718,90,0.0,9662,3,3,5,4,1977,1977,0.0,...,0,0,0,2006,29,29,0,0,1,0
2414,60,58.0,17104,3,3,7,5,2006,2006,0.0,...,0,0,0,2006,0,0,554,0,1,0
1989,30,60.0,8520,4,3,5,6,1923,2006,0.0,...,0,0,0,2007,84,1,0,0,0,0
625,20,0.0,9500,3,3,6,5,1963,1963,247.0,...,0,0,0,2009,46,46,609,1,0,0


In [14]:
#The number of columns on the scores I'm using and the test data are the same.
clean_test.columns

Index(['ms_subclass', 'lot_frontage', 'lot_area', 'lot_shape', 'land_slope',
       'overall_qual', 'overall_cond', 'year_built', 'year_remod/add',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'bsmt_qual', 'bsmt_cond',
       'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2',
       'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf', 'heating_qc',
       'central_air', 'electrical', '1st_flr_sf', '2nd_flr_sf',
       'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath', 'bsmt_half_bath',
       'full_bath', 'half_bath', 'bedroom_abvgr', 'kitchen_abvgr',
       'kitchen_qual', 'totrms_abvgrd', 'functional', 'fireplaces',
       'fireplace_qu', 'garage_finish', 'garage_cars', 'garage_area',
       'garage_qual', 'garage_cond', 'paved_drive', 'wood_deck_sf',
       'open_porch_sf', 'enclosed_porch', '3ssn_porch', 'screen_porch',
       'pool_area', 'fence', 'misc_val', 'yr_sold', 'age_at_sale',
       'age_since_remod', 'bsmnt_fin_sf', 'sold_in_crisis', 'has_wood_deck',


In [34]:
clean_test.shape

(878, 60)

### Re-loading and Cleaning Data from Model Build

In [15]:
df = pd.read_csv('./datasets/clean_recoded_ames.csv')
df.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,mo_sold,yr_sold,sale_type,saleprice,age_at_sale,age_since_remod,bsmnt_fin_sf,sold_in_crisis,has_wood_deck,has_pool
0,109,533352170,60,RL,0.0,13517,Pave,NaN,3,Lvl,...,3,2010,WD,130500,34,5,533.0,1,0,0
1,544,531379050,60,RL,43.0,11492,Pave,NaN,3,Lvl,...,4,2009,WD,220000,13,12,637.0,1,0,0
2,153,535304180,20,RL,68.0,7922,Pave,NaN,4,Lvl,...,1,2010,WD,109000,57,3,731.0,1,0,0
3,318,916386060,60,RL,73.0,9802,Pave,NaN,4,Lvl,...,4,2010,WD,174000,4,3,0.0,1,1,0
4,255,906425045,50,RL,82.0,14235,Pave,NaN,3,Lvl,...,3,2010,WD,138500,110,17,0.0,1,0,0


In [16]:
df['mo_sold'] = df['mo_sold'].astype(str)

In [17]:
df['garage_type'].fillna('None', inplace = True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049 entries, 0 to 2048
Data columns (total 87 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2049 non-null   int64  
 1   pid              2049 non-null   int64  
 2   ms_subclass      2049 non-null   int64  
 3   ms_zoning        2049 non-null   object 
 4   lot_frontage     2049 non-null   float64
 5   lot_area         2049 non-null   int64  
 6   street           2049 non-null   object 
 7   alley            140 non-null    object 
 8   lot_shape        2049 non-null   int64  
 9   land_contour     2049 non-null   object 
 10  utilities        2049 non-null   object 
 11  lot_config       2049 non-null   object 
 12  land_slope       2049 non-null   int64  
 13  neighborhood     2049 non-null   object 
 14  condition_1      2049 non-null   object 
 15  condition_2      2049 non-null   object 
 16  bldg_type        2049 non-null   object 
 17  house_style   

In [19]:
numeric_data = df.select_dtypes(exclude = 'object')
numeric_data.head(5)

,id,pid,ms_subclass,lot_frontage,lot_area,lot_shape,land_slope,overall_qual,overall_cond,year_built,...,fence,misc_val,yr_sold,saleprice,age_at_sale,age_since_remod,bsmnt_fin_sf,sold_in_crisis,has_wood_deck,has_pool
0,109,533352170,60,0.0,13517,3,3,6,8,1976,...,0,0,2010,130500,34,5,533.0,1,0,0
1,544,531379050,60,43.0,11492,3,3,7,5,1996,...,0,0,2009,220000,13,12,637.0,1,0,0
2,153,535304180,20,68.0,7922,4,3,5,7,1953,...,0,0,2010,109000,57,3,731.0,1,0,0
3,318,916386060,60,73.0,9802,4,3,5,5,2006,...,0,0,2010,174000,4,3,0.0,1,1,0
4,255,906425045,50,82.0,14235,3,3,6,8,1900,...,0,0,2010,138500,110,17,0.0,1,0,0


In [20]:
numeric_data.dtypes

id                   int64
pid                  int64
ms_subclass          int64
lot_frontage       float64
lot_area             int64
lot_shape            int64
land_slope           int64
overall_qual         int64
overall_cond         int64
year_built           int64
year_remod/add       int64
mas_vnr_area       float64
exter_qual           int64
exter_cond           int64
bsmt_qual            int64
bsmt_cond            int64
bsmt_exposure        int64
bsmtfin_type_1       int64
bsmtfin_sf_1       float64
bsmtfin_type_2       int64
bsmtfin_sf_2       float64
bsmt_unf_sf        float64
total_bsmt_sf      float64
heating_qc           int64
central_air          int64
electrical           int64
1st_flr_sf           int64
2nd_flr_sf           int64
low_qual_fin_sf      int64
gr_liv_area          int64
bsmt_full_bath     float64
bsmt_half_bath     float64
full_bath            int64
half_bath            int64
bedroom_abvgr        int64
kitchen_abvgr        int64
kitchen_qual         int64
t

In [40]:
numeric_data_cols = numeric_data.columns.tolist()
numeric_data_cols

['id',
 'pid',
 'ms_subclass',
 'lot_frontage',
 'lot_area',
 'lot_shape',
 'land_slope',
 'overall_qual',
 'overall_cond',
 'year_built',
 'year_remod/add',
 'mas_vnr_area',
 'exter_qual',
 'exter_cond',
 'bsmt_qual',
 'bsmt_cond',
 'bsmt_exposure',
 'bsmtfin_type_1',
 'bsmtfin_sf_1',
 'bsmtfin_type_2',
 'bsmtfin_sf_2',
 'bsmt_unf_sf',
 'total_bsmt_sf',
 'heating_qc',
 'central_air',
 'electrical',
 '1st_flr_sf',
 '2nd_flr_sf',
 'low_qual_fin_sf',
 'gr_liv_area',
 'bsmt_full_bath',
 'bsmt_half_bath',
 'full_bath',
 'half_bath',
 'bedroom_abvgr',
 'kitchen_abvgr',
 'kitchen_qual',
 'totrms_abvgrd',
 'functional',
 'fireplaces',
 'fireplace_qu',
 'garage_yr_blt',
 'garage_finish',
 'garage_cars',
 'garage_area',
 'garage_qual',
 'garage_cond',
 'paved_drive',
 'wood_deck_sf',
 'open_porch_sf',
 'enclosed_porch',
 '3ssn_porch',
 'screen_porch',
 'pool_area',
 'fence',
 'misc_val',
 'yr_sold',
 'saleprice',
 'age_at_sale',
 'age_since_remod',
 'bsmnt_fin_sf',
 'sold_in_crisis',
 'has_wood

In [21]:
cols_to_drop = ['id', 'pid', 'saleprice', 'garage_yr_blt']
X_numeric = numeric_data.drop(cols_to_drop, axis = 1)
y_numeric = df['saleprice']

In [22]:
X_numeric.shape, y_numeric.shape

((2049, 60), (2049,))

In [23]:
X_numeric_train, X_numeric_test, y_numeric_train, y_numeric_test = train_test_split(X_numeric, y_numeric)

In [24]:
X_numeric_train.shape, X_numeric_test.shape, y_numeric_train.shape, y_numeric_test.shape

((1536, 60), (513, 60), (1536,), (513,))

In [25]:
lr_numeric = LinearRegression()

In [26]:
lr_numeric.fit(X_numeric_train, y_numeric_train)

LinearRegression()

In [27]:
lr_numeric.score(X_numeric_train, y_numeric_train)

0.8991195649291944

In [28]:
lr_numeric.score(X_numeric_test, y_numeric_test)

0.8848910676561658

In [29]:
y_preds = lr_numeric.predict(clean_test)

In [30]:
y_preds

array([150174.44777851, 151290.9987916 , 213115.40926633, 109443.00509386,
       184454.0232037 ,  72345.04436891,  95835.21915485, 142495.4844471 ,
       185627.56536792, 155594.78921326, 164681.72628186, 112164.44552359,
       156072.32066006, 251584.19663014, 165312.86060799, 121833.72662641,
       144523.65205886, 115556.58958962, 195049.65728181, 210565.16524762,
       140997.6072609 , 118278.67928159, 203557.19411894, 172701.20409445,
       194084.72227411, 115079.07095483, 115589.19306497, 153013.61659139,
       162693.61927773,  44250.07353633, 102517.8991271 ,  88188.08985975,
       265954.22335469, 160009.43482988, 229908.47842615, 172850.50709178,
       127900.67584449,  67097.38617465, 123760.51743432, 196903.80943373,
       158505.53660031, 213765.15568418, 155092.40043798, 182039.55014103,
       239926.63774471,  88798.72229382, 238992.4293716 , 113632.30148095,
       117500.95985442, 114959.97817626, 116335.34326104, 222375.33910737,
       258335.31238657, 1

In [31]:
y_preds.shape

(878,)

In [37]:
type(y_preds)

numpy.ndarray

In [38]:
clean_test['SalePrice'] = y_preds

In [39]:
clean_test

,ms_subclass,lot_frontage,lot_area,lot_shape,land_slope,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,...,fence,misc_val,yr_sold,age_at_sale,age_since_remod,bsmnt_fin_sf,sold_in_crisis,has_wood_deck,has_pool,SalePrice
id,,,,,,,,,,,,,,,,,,,,,
2658,190,69.0,9142,4,3,6,8,1910,1950,0.0,...,0,0,2006,96,56,0,0,0,0,150174.447779
2718,90,0.0,9662,3,3,5,4,1977,1977,0.0,...,0,0,2006,29,29,0,0,1,0,151290.998792
2414,60,58.0,17104,3,3,7,5,2006,2006,0.0,...,0,0,2006,0,0,554,0,1,0,213115.409266
1989,30,60.0,8520,4,3,5,6,1923,2006,0.0,...,0,0,2007,84,1,0,0,0,0,109443.005094
625,20,0.0,9500,3,3,6,5,1963,1963,247.0,...,0,0,2009,46,46,609,1,0,0,184454.023204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,60,80.0,8000,4,3,6,6,1974,1974,0.0,...,0,0,2007,33,33,1084,0,0,0,193457.659279
1234,60,90.0,14670,4,3,6,7,1966,1999,410.0,...,3,0,2008,42,9,575,1,0,0,236931.493321
1373,20,55.0,8250,4,3,5,5,1968,1968,0.0,...,0,0,2008,40,40,742,1,0,0,121087.799494


In [40]:
clean_test['SalePrice'].max(), clean_test['SalePrice'].min(), clean_test['SalePrice'].median(), clean_test['SalePrice'].mean()

(546742.0767442612, 869.3651307316031, 163924.47966110858, 180186.85943641906)

In [46]:
submission = clean_test[['SalePrice']]
submission

,SalePrice
id,
2658,150174.447779
2718,151290.998792
2414,213115.409266
1989,109443.005094
625,184454.023204
...,...
1662,193457.659279
1234,236931.493321
1373,121087.799494


In [47]:
submission.to_csv('./datasets/DK_submission.csv')